In [41]:

def create_equity_curve(df,z0,z1,pos):
    ec=df.diff(axis=0)
    ec.iloc[0]=0
    
    i=1
    while i<len(ec):
        ec.iloc[i]=ec.iloc[i-1]+ec.iloc[i]*z0*z1*pos
        i+=1
    return ec
    
    

def pos():
    pos=1
    while True:
        a=raw_input("Please input 'y' if position is long and 'n' if position is short: ").lower()
    
        if a=='y':
            break
        if a=='n':
            pos=pos*(-1)
            break
        else:
            print "Please input either 'y' or 'n' for position"
    return pos

def z_factor(product1,product2):
    z=1
    factor=1000
    if product1=='RBOB' and product2 =='RBOB':
        z=42
    elif product1=='HO' and product2 =='HO':
        z=42
    elif product1=='RBOB' and product2 =='HO':
        z=42
    elif product1=='HO' and product2 =='RBOB':
        z=42
    elif product1=='Gasoil' and product2 =='Gasoil':
        factor=100
    elif product1=='Gasoil' and product2 =='Brent':
        z=(1/7.45)
    
    else:
        z=1
        factor=1000
    
    return z,factor
        
def create_sprd(dataframe1, dataframe2):
   
    dataframe1.reset_index(inplace=True)
    dataframe2.reset_index(inplace=True)
    date=dataframe1['Date']
    
    dataframe1.drop(columns=['Date'],inplace=True)
    dataframe2.drop(columns=['Date'],inplace=True)
    
    while True:
        sprd_type=int(raw_input('Please indicate if this is a 1) Crack or 2) Spread - please input integer 1 or 2: '))
        
        if sprd_type==1:
            
            new_df=dataframe1*42-dataframe2
            new_df['Date']=date
            new_df.set_index('Date',inplace=True)
            new_df.fillna(method='bfill',inplace=True)
            new_df.fillna(method='ffill',inplace=True)
            
            return new_df
            break
        
        elif sprd_type==2:
            
            new_df= dataframe1-dataframe2
            new_df['Date']=date
            new_df.set_index('Date',inplace=True)
            new_df.fillna(method='bfill',inplace=True)
            new_df.fillna(method='ffill',inplace=True)
            
            return new_df
            break
        
        else:
            print 'You have inputted incorrectly. Please try again and enter either integer 1 or 2.'
    

class Mth_1(object):
    
    def __init__(self):
        pass
        
    def pdt_type1(self):
        
        product=''
        while True:
            inst=int(raw_input('Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: '))
    
            if inst==1:
                product='Brent'
                break
            elif inst==2:
                product='Gasoil'
                break   
            elif inst==3:
                product='HO'
                break      
            elif inst==4:
                product='RBOB'
                break       
            elif inst==5:
                product='WTI'
                break       
            else:
                print 'You did not input a correct number. Please try again.'
        
        return product
        
    
    def mth_no1(self):
        month=''
        while True:
            Mth=int(raw_input('Please input the month of the contract which you wish to evaluate in numbers from 1-12: '))
    
            if Mth==1:
                month='Jan'
                break
            elif Mth==2:
                month='Feb'
                break   
            elif Mth==3:
                month='Mar'
                break   
            elif Mth==4:
                month='Apr'
                break   
            elif Mth==5:
                month='May'
                break          
            elif Mth==6:
                month='Jun'
                break   
            elif Mth==7:
                month='Jul'
                break   
            elif Mth==8:
                month='Aug'
                break   
            elif Mth==9:
                month='Sep'
                break   
            elif Mth==10:
                month='Oct'
                break   
            elif Mth==11:
                month='Nov'
                break   
            elif Mth==12:
                month='Dec'
                break 
            else:
                print 'You did not input a correct number. Please try again.'
        
        return month

    def dataframe_create(self,start_mth,start_day,end_mth,end_day,i):

        count=0
        path = os.getcwd()
        
        listing = os.listdir(path)
        for infile in listing:
            temp=pd.read_excel(infile) #read each file into panda dataframe
    
            contract_yr_str=infile[-8:-4] # get the contract year
    
            seasonal_year=int(contract_yr_str)-1 # get the seasonal year to create the index - prev year
            seasonal_year=str(seasonal_year) # convert to string

            start_date=str(int(seasonal_year)+i)+'-'+start_mth+'-'+start_day # create start date
            end_date=str(int(seasonal_year)+j)+'-'+end_mth+'-'+end_day # create end date
            
            #Join a date dataframe and the temp dataframe together
            
            dates=pd.date_range(start_date, end_date) # range of dates for date axis
            dates=dates.strftime('%Y-%m-%d') #make sure no left out dates
            sample_data=np.arange(0,len(dates)) #just sample numbers to create dataframe
    
            dates_dataframe=pd.DataFrame({'Date':dates,'data':sample_data}) # create dates dataframe
            dates_dataframe=dates_dataframe.set_index('Date') # set index as Date to facilitate joining
    
            temp=temp.set_index('Date') # set index as Date to facilitate joining
    
            temp=dates_dataframe.join(temp) # join dataframes together on Date
               
            #filter dates by the start and end date
            temp.reset_index(inplace=True)
    
            temp=temp[temp['Date']>=start_date] # filter for start date

            temp=temp[temp['Date']<=end_date] # filter for end date

            temp['Date']=pd.to_datetime(temp['Date']).dt.strftime('%m/%d') # putting date in mm/dd form
            temp=temp.set_index('Date') # putting date back as index
            temp=pd.DataFrame(temp['Settle'],columns=['Settle']) # setting dataframe with date and settle only

    
            if count==0:
                Seasonal_Combi=temp
                seasonal_year=str(int(seasonal_year)+1)
                Seasonal_Combi.rename(columns={'Settle':seasonal_year},inplace=True)
        
                count+=1
        
            else:
    
                seasonal_year=str(int(seasonal_year)+1)
                Seasonal_Combi['Settle']=temp['Settle']
                Seasonal_Combi.rename(columns={'Settle':seasonal_year},inplace=True)

        #don't need this, i believe
        #Seasonal_Combi.drop(columns=['Open','High','Low','Last','Change', 'Volume','Prev. Day Open Interest'],inplace=True)   

        
        return Seasonal_Combi
            
            

class Mth_2(object):
    
    def __init__(self):
        pass
        
    def pdt_type2(self):
        
        product=''
        while True:
            inst=int(raw_input('Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: '))
    
            if inst==1:
                product='Brent'
                break
            elif inst==2:
                product='Gasoil'
                break   
            elif inst==3:
                product='HO'
                break      
            elif inst==4:
                product='RBOB'
                break       
            elif inst==5:
                product='WTI'
                break       
            else:
                print 'You did not input a correct number. Please try again.'
        
        return product
        
    
    def mth_no2(self):
        month=''
        while True:
            Mth=int(raw_input('Please input the month of the contract which you wish to evaluate in numbers from 1-12: '))
    
            if Mth==1:
                month='Jan'
                break
            elif Mth==2:
                month='Feb'
                break   
            elif Mth==3:
                month='Mar'
                break   
            elif Mth==4:
                month='Apr'
                break   
            elif Mth==5:
                month='May'
                break          
            elif Mth==6:
                month='Jun'
                break   
            elif Mth==7:
                month='Jul'
                break   
            elif Mth==8:
                month='Aug'
                break   
            elif Mth==9:
                month='Sep'
                break   
            elif Mth==10:
                month='Oct'
                break   
            elif Mth==11:
                month='Nov'
                break   
            elif Mth==12:
                month='Dec'
                break 
            else:
                print 'You did not input a correct number. Please try again.'
        
        return month

    def dataframe_create(self, start_mth, start_day, end_mth,end_day,i):

        count=0
        path = os.getcwd()

        listing = os.listdir(path)
        for infile in listing:
            temp=pd.read_excel(infile) #read each file into panda dataframe
    
            contract_yr_str=infile[-8:-4] # get the contract year
    
            seasonal_year=int(contract_yr_str)-1 # get the seasonal year to create the index - prev year
            seasonal_year=str(seasonal_year) # convert to string
     
            start_date=str(int(seasonal_year)+i)+'-'+start_mth+'-'+start_day # create start date
            end_date=str(int(seasonal_year)+j)+'-'+end_mth+'-'+end_day # create end date
            
            #Join a date dataframe and the temp dataframe together
            
            dates=pd.date_range(start_date, end_date) # range of dates for date axis
            dates=dates.strftime('%Y-%m-%d') #make sure no left out dates
            sample_data=np.arange(0,len(dates)) #just sample numbers to create dataframe
    
            dates_dataframe=pd.DataFrame({'Date':dates,'data':sample_data}) # create dates dataframe
            dates_dataframe=dates_dataframe.set_index('Date') # set index as Date to facilitate joining
    
            temp=temp.set_index('Date') # set index as Date to facilitate joining
    
            temp=dates_dataframe.join(temp) # join dataframes together on Date
               
            #filter dates by the start and end date
            temp.reset_index(inplace=True)
    
            temp=temp[temp['Date']>=start_date] # filter for start date

            temp=temp[temp['Date']<=end_date] # filter for end date

            temp['Date']=pd.to_datetime(temp['Date']).dt.strftime('%m/%d') # putting date in mm/dd form
            temp=temp.set_index('Date') # putting date back as index
            temp=pd.DataFrame(temp['Settle'],columns=['Settle']) # setting dataframe with date and settle only
    
            if count==0:
                Seasonal_Combi=temp
                seasonal_year=str(int(seasonal_year)+1)
                Seasonal_Combi.rename(columns={'Settle':seasonal_year},inplace=True)
        
                count+=1
        
            else:
    
                seasonal_year=str(int(seasonal_year)+1)
                Seasonal_Combi['Settle']=temp['Settle']
                Seasonal_Combi.rename(columns={'Settle':seasonal_year},inplace=True)

        #Seasonal_Combi.drop(columns=['Open','High','Low','Last','Change','Volume','Prev. Day Open Interest'],inplace=True)   

        
        return Seasonal_Combi
    
    
import numpy as np
import pandas as pd
import os

from plotly import __version__
import cufflinks as cf
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
cf.go_offline()

%matplotlib inline

#user specify instrument and contract month

product1=''
month1=''
product2=''
month2=''

front_mth=Mth_1()
product1 = front_mth.pdt_type1()
month1=front_mth.mth_no1()

back_mth=Mth_2()
product2=back_mth.pdt_type2()
month2=back_mth.mth_no2()

        

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: 4
Please input the month of the contract which you wish to evaluate in numbers from 1-12: 7
Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: 5
Please input the month of the contract which you wish to evaluate in numbers from 1-12: 7


In [42]:
cd Quandl

C:\Users\boonsin.teo\Documents\Quandl


In [43]:
cd $product1

C:\Users\boonsin.teo\Documents\Quandl\RBOB


In [44]:
cd $month1

C:\Users\boonsin.teo\Documents\Quandl\RBOB\Jul


In [45]:
start_mth=raw_input('Please input start mth of seasonal period in MM format: ')
start_day=raw_input('Please input start day of seasonal period in DD format: ')
i=int(raw_input('Please input 1 if start mth is the same as contract year and 0 if not: '))
            
end_mth=raw_input('Please input end mth of seasonal period in MM format: ')
end_day=raw_input('Please input end day of seasonal period in DD format: ')
            
j=int(raw_input('Please input 1 if end mth is the same as contract year and 0 if not: '))


front_month=front_mth.dataframe_create(start_mth,start_day,end_mth,end_day,i)

Please input start mth of seasonal period in MM format: 01
Please input start day of seasonal period in DD format: 07
Please input 1 if start mth is the same as contract year and 0 if not: 1
Please input end mth of seasonal period in MM format: 03
Please input end day of seasonal period in DD format: 29
Please input 1 if end mth is the same as contract year and 0 if not: 1


In [46]:
cd ..

C:\Users\boonsin.teo\Documents\Quandl\RBOB


In [47]:
cd ..

C:\Users\boonsin.teo\Documents\Quandl


In [48]:
cd ..

C:\Users\boonsin.teo\Documents


In [49]:
cd Quandl

C:\Users\boonsin.teo\Documents\Quandl


In [50]:
cd $product2

C:\Users\boonsin.teo\Documents\Quandl\WTI


In [51]:
cd $month2

C:\Users\boonsin.teo\Documents\Quandl\WTI\Jul


In [52]:
back_month=back_mth.dataframe_create(start_mth, start_day,end_mth,end_day,i)

In [53]:
cd ..

C:\Users\boonsin.teo\Documents\Quandl\WTI


In [54]:
cd ..

C:\Users\boonsin.teo\Documents\Quandl


In [55]:
cd ..

C:\Users\boonsin.teo\Documents


In [56]:
front_month.head()

,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
Date,,,,,,,,,,,,,,,,,,,,,
01/07,NaN,0.6841,0.5449,0.4413,0.6635,NaN,0.6804,0.8438,0.9663,1.2787,...,2.5776,1.3324,2.2698,2.5005,NaN,2.8432,2.7989,1.6187,1.3976,NaN
01/08,NaN,0.6900,0.5490,0.4380,NaN,0.8007,0.6713,0.8506,0.9842,NaN,...,2.6169,1.3407,2.2790,NaN,NaN,2.8682,2.7863,1.6297,1.3823,NaN
01/09,0.5612,0.6807,0.5445,NaN,NaN,0.7921,0.6543,0.8698,0.9945,NaN,...,2.5760,1.3632,NaN,NaN,2.8521,2.8637,2.7759,1.6273,NaN,1.7778
01/10,0.5597,0.6830,NaN,NaN,0.6748,0.8378,0.6639,0.8512,NaN,1.2754,...,2.5210,NaN,NaN,2.5474,2.8525,2.8757,2.7949,NaN,NaN,1.7538
01/11,0.5420,NaN,NaN,0.4438,0.6880,0.8270,0.6516,NaN,NaN,1.2818,...,2.4966,NaN,2.2696,2.5854,2.8442,2.8422,NaN,NaN,1.3510,1.7948


In [57]:
back_month.head()

,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Date,,,,,,,,,,,,,,,,,,,,,
01/07,NaN,22.92,17.75,13.59,21.79,NaN,21.78,27.04,31.28,44.37,...,54.41,85.20,92.07,NaN,94.67,92.16,51.95,38.32,NaN,NaN
01/08,NaN,23.33,17.85,13.54,NaN,24.85,21.57,27.05,31.61,NaN,...,54.09,85.53,NaN,NaN,94.81,91.07,52.19,38.27,NaN,NaN
01/09,18.17,23.12,17.63,NaN,NaN,25.13,20.91,27.60,31.82,NaN,...,54.13,NaN,NaN,102.35,94.87,90.69,52.08,NaN,55.43,NaN
01/10,18.06,22.98,NaN,NaN,22.17,26.20,21.27,27.33,NaN,44.20,...,NaN,NaN,93.76,103.03,95.33,91.59,NaN,NaN,54.27,NaN
01/11,17.47,NaN,NaN,13.77,22.77,26.18,20.79,NaN,NaN,44.51,...,NaN,85.24,95.32,101.85,94.98,NaN,NaN,36.55,55.56,NaN


In [58]:
Spread_Combi=create_sprd(front_month, back_month)

Please indicate if this is a 1) Crack or 2) Spread - please input integer 1 or 2: 1


In [59]:
Spread_Combi.dropna(axis=0,how='all',inplace=True) # drop rows with all NAs

In [60]:
Spread_Combi.dropna(axis=1,how='all',inplace=True)# drop columns with all NAs

In [61]:
position=pos()
print 'PnL will be calculated basis 1kb lot size for standardization purposes.'

Please input 'y' if position is long and 'n' if position is short: y
PnL will be calculated basis 1kb lot size for standardization purposes.


In [62]:
zf=z_factor(product1, product2)

In [63]:
#PnL Series
pnl_series=(Spread_Combi.iloc[-1]-Spread_Combi.iloc[0])*zf[0]*zf[1]*position

In [64]:
#Win %
Win_Percent=float(pnl_series[pnl_series>0].count())/float(pnl_series.count())*100

In [65]:
#Largest Drawdown - use max and min
max_series=Spread_Combi.max()
min_series=Spread_Combi.min()
if position==1:
    max_drawdown=(min_series-Spread_Combi.iloc[0])*position*zf[0]*zf[1]
else:
    max_drawdown=(max_series-Spread_Combi.iloc[0])*position*zf[0]*zf[1]

In [66]:
#% return - calculate using last value vs first value - similar to pnl calc
percent_return=((Spread_Combi.iloc[-1]-Spread_Combi.iloc[0])/(Spread_Combi.iloc[0]))

In [67]:
pnl_series

1996     1056.6
1997      -74.6
1998      721.6
1999      479.2
2000     1334.0
2001     1364.6
2002     1008.2
2003      826.4
2004     1359.0
2005     2973.8
2006     5152.2
2007     4548.0
2008    -5606.2
2009     6725.2
2010     1415.6
2011     7884.8
2012    14190.8
2013     5614.8
2014    -3910.8
2015     6506.0
2016     1707.0
2017       98.8
dtype: float64

In [68]:
percent_return

1996    0.195652
1997   -0.012835
1998    0.140504
1999    0.096914
2000    0.219516
2001    0.155432
2002    0.148335
2003    0.098386
2004    0.146057
2005    0.318551
2006    0.356485
2007    0.354482
2008   -0.376781
2009    4.336600
2010    0.139721
2011    0.608818
2012    0.813777
2013    0.226912
2014   -0.154006
2015    0.405727
2016    0.083762
2017    0.005136
dtype: float64

In [69]:
max_drawdown

1996    -426.6
1997    -810.2
1998     -18.8
1999    -727.6
2000    -264.4
2001   -1112.0
2002    -394.8
2003    -596.2
2004    -332.0
2005    -802.8
2006   -9607.6
2007    -385.8
2008   -9992.0
2009       0.0
2010    -352.0
2011    -289.4
2012    -663.2
2013   -1012.8
2014   -4535.2
2015    -300.2
2016   -5887.0
2017   -2315.8
dtype: float64

In [70]:
print 'the win percentage is %.2f percent' %Win_Percent

the win percentage is 86.36 percent


In [71]:
Equity_Curve=create_equity_curve(Spread_Combi, zf[0],zf[1],position)

In [72]:
Equity_Curve.iplot(kind='line')